# Agregar tokens de conexto especiales

Modificar el  vocabulario  y  el  tokenizador  que  implementamos  en  la  sección  anterior,  SimpleTokenizerV2,  para  admitir  dos  nuevos  tokens,  <|unk|>  y  <|endoftext|>

In [1]:
import re
with open("../txt/The_Verdict.txt", "r", encoding="utf-8") as f:
    r_text = f.read()

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',r_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

![Texto alternativo](./imgs/2.7.png)

Se puede  modificar  el  tokenizador  para  que  use  un  token  <|unk|>  si  encuentra  una  palabra  que  no  forma  parte  del  vocabulario.  Además,  añadimos  un  token  entre  textos  no  relacionados.  Por  ejemplo,  al  entrenar  LLMs  tipo  GPT  con  varios  documentos  o  libros  independientes,  es  habitual  insertar  un  token  antes  de  cada  documento  o  libro  que  sigue  a  una  fuente  de  texto  anterior,  

In [8]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"]) #Ampliación del vocabulario con dos tokens más
vocab = {w:i for i, w in enumerate(all_tokens)}
print(len(vocab))

#Comprobaciones
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)


1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


![Texto alternativo](./imgs/2.8.png)

Al  trabajar  con  múltiples  fuentes  de  texto  independientes,  añadimos  tokens  <|endoftext|>  entre  estos  textos.  Estos  tokens  <|endoftext|>  actúan  como  marcadores,  señalando  el  inicio  o  el  final  de  un  segmento  específico,  lo  que  permite  un  procesamiento  y  una  comprensión  más  eficaces  por  parte  del  LLM.

In [11]:
#A parir de la salida anterior se pueden confirmar los dos tokens añadidos
#Ajutar el tokeninzador
class SimpleTokenizerV2:
    def __init__(self, vocab):   
        self.vocab = vocab #Almacenar el vocabulario para acceder a el en los métodos 
        self.reverse_vocab = {i:w for w, i in self.vocab.items()} #Vocabulario inverso para el decode, asigna los identificadores a los tokens

    #Procesar el texto de entrada en identificadores de tokens
    def encode(self, real_text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',real_text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.vocab else  "<|unk|>" for item in preprocessed ] #reemplaza  palabras  desconocidas  por  tokens  <|unk|>

        ids = [self.vocab[p] for p in preprocessed]
        return ids
    
    #Convertir los ids de token en texto
    def decode(self, ids):
        text = [self.reverse_vocab[id] for id in ids]
        text = " ".join(text)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #Reemplazar espacios antes de la puntuación especificada 
        return text

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab=vocab)
ids = tokenizer.encode(text)
text = tokenizer.decode(ids)

print(ids)
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Hasta  ahora,  se ha analizado  la  tokenización  como  un  paso  esencial  en  el  procesamiento  de  texto  como  entrada  para  los  LLM.  Dependiendo  del  LLM,  algunos  investigadores  también  consideran  tokens  especiales  adicionales,  como  los  siguientes:
- [BOS]  (inicio  de  secuencia):  Este  token  marca  el  inicio  de  un  texto.  Indica  al  LLM  dónde  comienza  un  fragmento  de  contenido.
- [EOS]  (fin  de  secuencia):  Este  token  se  coloca  al  final  de  un  texto  y  es  especialmente  útil  al  concatenar  varios  textos  no  relacionados,  similar  a  <|endoftext|>.  Por  ejemplo,  al  combinar  dos  artículos  o  libros  diferentes  de  Wikipedia,  el  token  [EOS]  indica  dónde  termina  un  artículo  y  dónde  empieza  el  siguiente.
- [PAD]  (relleno):  Al  entrenar  LLM  con  lotes  mayores  a  uno,  estos  pueden  contener  textos  de  longitud  variable.  Para  garantizar  que  todos  los  textos  tengan  la  misma  longitud,  los  textos  más  cortos  se  amplían  o  se  rellenan  con  el  token  [PAD] ,  hasta  la  longitud  del  texto  más  largo  del  lote.

[Agregar tokens de contexto especiales](./4_codificacion_bytePair.ipynb)